In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import pickle
from wordcloud import WordCloud
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import math
from time import time
from underthesea import word_tokenize
from operator import itemgetter
import warnings
import jieba
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('2k_data_Sep.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Feedback ID      2000 non-null   float64
 1   Lookup Text      2000 non-null   object 
 2   Question Text    2000 non-null   object 
 3   Feedback Type    2000 non-null   object 
 4   Feedback Result  2000 non-null   object 
 5   Human Feedback   2000 non-null   object 
 6   Lookup Image     2000 non-null   object 
 7   Question Image   2000 non-null   object 
dtypes: float64(1), object(7)
memory usage: 125.1+ KB


In [3]:
dup_row = df['Lookup Text'].duplicated().sum()
dup_row

43

In [4]:
df = df.drop_duplicates().reset_index(drop = True)
df.head()

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
0,4416.0,b) \( \left(\frac{-4}{15}-\frac{18}{19}\right)...,a) \( \frac{2}{3}+\frac{-1}{3}+\frac{7}{15} \)...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/9a7aa76...,https://s3.icankid.io/uploads/question/41bf56d...
1,4419.0,Đề số \( 1 . \)\n1. Tính\nb) \( \sqrt{25} \)\n...,b) \( \sqrt{25} \)\nc) \( \sqrt{-36} \)\na) \(...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/0398020...,https://s3.icankid.io/uploads/question/5a47b89...
2,4420.0,1. Tính: \( \sqrt{\frac{25}{4}}+\left(\sqrt{\f...,Đề số \( 3 . \)\n1. Tính: \( \sqrt{\frac{25}{4...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/a81b075...,https://s3.icankid.io/uploads/question/2d3358a...
3,4423.0,Câu 49: Cho hàm số \( y=f(x) \) có đạo hàm là ...,Câu 49. Cho hàm số \( y=f(x) \) có đạo hàm \( ...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/a981260...,https://s3.icankid.io/uploads/question/aaa4976...
4,4425.0,b. Giải phương trình: \( 2 \sin ^{2} x+\sqrt{3...,1) \( \sin ^{2} 2 x-(2+\sqrt{3}) \sin 2 x+\sqr...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/caa519f...,https://s3.icankid.io/uploads/question/7ea6f5f...


In [5]:
mask1 = df['Human Feedback'] == 'MATCH'

df_pos = df[mask1].reset_index(drop = True)
df_pos

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
0,4419.0,Đề số \( 1 . \)\n1. Tính\nb) \( \sqrt{25} \)\n...,b) \( \sqrt{25} \)\nc) \( \sqrt{-36} \)\na) \(...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/0398020...,https://s3.icankid.io/uploads/question/5a47b89...
1,4420.0,1. Tính: \( \sqrt{\frac{25}{4}}+\left(\sqrt{\f...,Đề số \( 3 . \)\n1. Tính: \( \sqrt{\frac{25}{4...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/a81b075...,https://s3.icankid.io/uploads/question/2d3358a...
2,4432.0,25. Tìm giá trị x (làm tròn đến chữ số thập ph...,b) Canh BC.\n5. Tìm giá trị \( x \) (làm tròn ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/71b3a3c...,https://s3.icankid.io/uploads/question/3c3ec9e...
3,4443.0,Câu 6. Cho \( \frac{x-6}{1998}+\frac{x-4}{2000...,Câu 6. Cho \( \frac{x-6}{1998}+\frac{x-4}{2000...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/690df82...,https://s3.icankid.io/uploads/question/44a094e...
4,4446.0,Câu 7. Các giá trị của \( x \) thỏa mã \( \lef...,Câu 7. Các giá trị của \( x \) thỏa mãn \( \le...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/539c341...,https://s3.icankid.io/uploads/question/eb404dc...
...,...,...,...,...,...,...,...,...
1016,9955.0,3.36. Cho biết tích của hai số tự nhiên \( n \...,3.36. Cho biết tích của hai số tự nhiên \( n \...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/a08206d...,https://s3.stag.icankids.com.vn/uploads/questi...
1017,9967.0,"Câu 10: Trong hệ trục Oxy, cho \( \vec{u}=(-2 ...","Câu 18: Trong mặt phẳng tọa độ \( O x y \), ch...",NEGATIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/d34ea5b...,https://s3.icankid.io/uploads/question/d44ded4...
1018,9973.0,6. Hai đoạn ống nước có chiều dài lần lượt là ...,6. Hai đoạn ống nước có chiều dài lần lượt là ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/f60a06b...,https://s3.icankid.io/uploads/question/2740c6d...
1019,9978.0,"bao nhiêu phần kế hoạch?\n8. Vào tháng 6, giá ...","8. Vào tháng 6 , giá niêm yết một chiếc ti vi ...",POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/9fee271...,https://s3.icankid.io/uploads/question/9363c3c...


In [6]:
def clean_function(text):  
    #lowercase
    text = text.lower()
    
#     #xoá ký tự đặc biệt, ký tự không phải alphanumeric 
#     text = re.sub("[^a-z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễòóọỏõôồốộổỗơờớợởỡìíịỉĩùúụủũưừứựửữỳýỵỷỹđ]",' ', text)
    
#     #xoá leading zero
#     text = [e.lstrip('0') for e in text.split(' ')]
#     text = (' ').join(text)
    
    #replace extra spaces with single space
    text = re.sub(' +',' ',text)
    
#     #xoá khoảng trắng ở đầu và cuối mỗi câu
#     text = text.strip()
    
    #word tokenzie
    text = word_tokenize(text, format='text')
    return text

In [7]:
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    vn_stop_words = file.read()
vn_stop_words = vn_stop_words.split('\n') 

# combine Vietnamese stopwords and English stopwords
stop_words = ENGLISH_STOP_WORDS.union(vn_stop_words)

In [8]:
df_pos['clean_question'] = df_pos['Question Text'].map(lambda x: clean_function(x))

In [9]:
question_ = [[text for text in x.split()] for x in df_pos['clean_question']]

In [10]:
len(question_)

1021

In [14]:
question_[0]

['b',
 ')',
 '\\',
 '(',
 '\\',
 'sqrt',
 '{',
 '25',
 '}_\\',
 ')',
 'c',
 ')',
 '\\',
 '(',
 '\\',
 'sqrt',
 '{',
 '-',
 '36',
 '}_\\',
 ')',
 'a',
 ')',
 '\\',
 '(',
 '\\',
 'sqrt',
 '{',
 '64',
 '}_\\',
 ')',
 '\\',
 '(',
 '0_=_\\_frac',
 '{',
 '2',
 's',
 '}_{_\\',
 'partial',
 's',
 '}_\\',
 ')',
 'e',
 ')',
 '\\',
 '(',
 '\\_sqrt',
 '{',
 '(',
 '-',
 '5',
 ')',
 '^',
 '{',
 '2',
 '}_}_\\',
 ')',
 'd',
 ')',
 '\\',
 '(',
 '\\',
 'sqrt',
 '{',
 '5',
 '^',
 '{',
 '2',
 '}_}_\\',
 ')',
 'f',
 ')',
 '\\',
 '(',
 '\\_frac',
 '{',
 '2',
 '}',
 '{',
 '3_}_\\',
 'sqrt',
 '{',
 '81',
 '}',
 '-',
 '\\_left',
 '(',
 '-',
 '\\',
 'frac',
 '{',
 '3',
 '}_{',
 '4',
 '}_\\_right',
 ')',
 ':',
 '\\',
 'sqrt',
 '{_\\',
 'frac',
 '{',
 '9',
 '}_{',
 '64',
 '}_}_+_\\_left',
 '(',
 '\\',
 'frac',
 '{_\\_sqrt',
 '{',
 '2',
 '}_}_{',
 '3_}_\\_right',
 ')',
 '^_{',
 '0',
 '}',
 '-',
 '(',
 '\\',
 'sqrt',
 '{',
 '3',
 '}',
 ')',
 '^',
 '{',
 '2',
 '}_\\',
 ')']

In [18]:
query = df_pos.iloc[0]['Lookup Text']
query = query.split()

In [20]:
dictionary = corpora.Dictionary(question_)

In [42]:
print(dictionary)

Dictionary<4149 unique tokens: ['(', ')', '-', '0', '0_=_\\_frac']...>


In [22]:
dictionary.token2id

{'(': 0,
 ')': 1,
 '-': 2,
 '0': 3,
 '0_=_\\_frac': 4,
 '2': 5,
 '25': 6,
 '3': 7,
 '36': 8,
 '3_}_\\': 9,
 '3_}_\\_right': 10,
 '4': 11,
 '5': 12,
 '64': 13,
 '81': 14,
 '9': 15,
 ':': 16,
 '\\': 17,
 '\\_frac': 18,
 '\\_left': 19,
 '\\_sqrt': 20,
 '^': 21,
 '^_{': 22,
 'a': 23,
 'b': 24,
 'c': 25,
 'd': 26,
 'e': 27,
 'f': 28,
 'frac': 29,
 'partial': 30,
 's': 31,
 'sqrt': 32,
 '{': 33,
 '{_\\': 34,
 '{_\\_sqrt': 35,
 '}': 36,
 '}_\\': 37,
 '}_\\_right': 38,
 '}_{': 39,
 '}_{_\\': 40,
 '}_}_+_\\_left': 41,
 '}_}_\\': 42,
 '}_}_{': 43,
 '.': 44,
 '1': 45,
 '16': 46,
 '1_._tính': 47,
 'cdot': 48,
 'số': 49,
 '}_+_\\_left': 50,
 '}_}': 51,
 '}_}_\\_right': 52,
 'đề': 53,
 ',': 54,
 '10': 55,
 'ba': 56,
 'bc': 57,
 'biết': 58,
 'canh': 59,
 'chỉ': 60,
 'chữ': 61,
 'giá_trị': 62,
 'hình': 63,
 'kích_thước': 64,
 'làm': 65,
 'mō': 66,
 'phân': 67,
 'ra': 68,
 'rằng': 69,
 'só': 70,
 'tam_giác_vuông': 71,
 'thâp': 72,
 'thú': 73,
 'trong': 74,
 'trên': 75,
 'tròn': 76,
 'tìm': 77,
 'với': 

In [23]:
feature_count = len(dictionary.token2id)
feature_count

4149

In [24]:
corpus = [dictionary.doc2bow(quest) for quest in question_]
corpus

[[(0, 11),
  (1, 17),
  (2, 5),
  (3, 1),
  (4, 1),
  (5, 6),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 2),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 15),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 3),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 3),
  (30, 1),
  (31, 2),
  (32, 7),
  (33, 16),
  (34, 1),
  (35, 1),
  (36, 4),
  (37, 5),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 2),
  (43, 1)],
 [(0, 5),
  (1, 5),
  (2, 4),
  (5, 5),
  (6, 1),
  (7, 2),
  (11, 2),
  (14, 1),
  (15, 1),
  (16, 2),
  (17, 9),
  (19, 1),
  (21, 3),
  (29, 4),
  (32, 4),
  (33, 9),
  (34, 3),
  (36, 5),
  (37, 1),
  (38, 1),
  (39, 3),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1)],
 [(0, 2),
  (1, 3),
  (12, 1),
  (16, 1),
  (17, 2),
  (24, 1),
  (44, 2),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),


In [25]:
tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_count)

In [26]:
query_vector = dictionary.doc2bow(query)
query_vector

[(44, 1), (45, 1), (49, 1)]

In [27]:
gensim = index[tfidf[query_vector]]

In [37]:
gensim

array([0.        , 0.09783565, 0.00334448, ..., 0.00343543, 0.00254754,
       0.00090751], dtype=float32)

In [41]:
index_ = gensim.argsort()[:5]
index